In [1]:
with open("input.txt", "r") as f:
    text = f.read()

In [2]:
len(text)

1115394

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

### Tokenize

In [4]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda x: [stoi[c] for c in x]
decode = lambda x: "".join([itos[i] for i in x])

In [5]:
encode("hi there")

[46, 47, 1, 58, 46, 43, 56, 43]

In [6]:
decode([46, 47, 1, 58, 46, 43, 56, 43])

'hi there'

#### Tensorizing tokens

In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
data.shape
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]

    print(f"{context}--->{target}")

tensor([18])--->47
tensor([18, 47])--->56
tensor([18, 47, 56])--->57
tensor([18, 47, 56, 57])--->58
tensor([18, 47, 56, 57, 58])--->1
tensor([18, 47, 56, 57, 58,  1])--->15
tensor([18, 47, 56, 57, 58,  1, 15])--->47
tensor([18, 47, 56, 57, 58,  1, 15, 47])--->58


this is the time dimension. another dimension is batch dimension. chunks of data processed in gpu independently

### Creating batches

In [10]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch(split="train")
print(f"inputs: {xb}")
print(f"targets: {yb}")

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b, t:t+1]

        print(f"input: {context.tolist()}--->target: {target.tolist()}")

inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
input: [24]--->target: [43]
input: [24, 43]--->target: [58]
input: [24, 43, 58]--->target: [5]
input: [24, 43, 58, 5]--->target: [57]
input: [24, 43, 58, 5, 57]--->target: [1]
input: [24, 43, 58, 5, 57, 1]--->target: [46]
input: [24, 43, 58, 5, 57, 1, 46]--->target: [43]
input: [24, 43, 58, 5, 57, 1, 46, 43]--->target: [39]
input: [44]--->target: [53]
input: [44, 53]--->target: [56]
input: [44, 53, 56]--->target: [1]
input: [44, 53, 56, 1]--->target: [58]
input: [44, 53, 56, 1, 58]--->target: [46]
input: [44, 53, 56, 1, 58, 46]--->target: [39]
input: [44, 53, 56, 1, 58, 46, 39]--->target: [58]
input: [44, 53, 56, 1, 58, 46, 39, 58]--->ta

### Creating layers

In [12]:
import torch
torch.manual_seed(1337)
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self,ix, targets):
        logits = self.embedding(ix)#B,T,C#batch, time, Channel(emb dim)
        return logits

model = BigramLanguageModel(vocab_size)
out = model(xb, yb)
out.shape


torch.Size([4, 8, 65])

#### Obtaining loss (wrong pytorch syntax way)

In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self,ix, targets):
        logits = self.embedding(ix)#B,T,C#batch, time, Channel(emb dim)

        loss = F.cross_entropy(logits, targets)
        return logits

model = BigramLanguageModel(vocab_size)
out = model(xb, yb)
out.shape

RuntimeError: Expected target size [4, 65], got [4, 8]

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss

Our dimension is B,T,C but torch expects B,C,T.

we had better pass using the shape of B*T, C instead of B,T,C

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self,ix, targets):
        logits = self.embedding(ix)#B,T,C#batch, time, Channel(emb dim)

        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

model = BigramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print(out.shape)
print(loss)

torch.Size([32, 65])
tensor(4.4971, grad_fn=<NllLossBackward0>)


We have 65 vocab size. so probability for each to be randomly selected is 1/65. so random loss should be -log(1/65)=4.1743872698956. Our loss is larger than that

#### Generate

In [15]:
import torch
torch.manual_seed(1337)
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self,ix, targets=None):
        logits = self.embedding(ix)#B,T,C#batch, time, Channel(emb dim)

        if targets==None:
            loss=None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_token):

        for _ in range(max_new_token):
            logits, _ = self(idx)
            # print(logits.shape)
            logits = logits[:, -1, :]
            # print(logits.shape)

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # print(idx_next.shape)
            idx = torch.cat((idx, idx_next),dim=1)
            
            # print(idx.shape)
        return idx

model = BigramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print(out.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
idx = model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_token=100)
idx

tensor([[ 0, 31, 56, 12, 55, 28,  7, 29, 35, 49, 58, 36, 53, 24,  4, 48, 24, 16,
         22, 45, 27, 24, 34, 64,  5, 30, 21, 53, 16, 55, 20, 42, 46, 57, 34,  4,
         60, 24, 24, 62, 39, 58, 48, 57, 41, 25, 54, 61, 24, 17, 30, 31, 28, 63,
         39, 53,  8, 55, 44, 64, 57,  3, 37, 57,  3, 64, 18,  7, 61,  6, 11, 43,
         17, 49, 64, 62, 48, 45, 15, 23, 18, 15, 46, 57,  2, 47, 35, 35,  8, 27,
         40, 64, 16, 52, 62, 13,  1, 25, 57,  3,  9]])

In [18]:
idx.shape #B,max token

torch.Size([1, 101])

In [19]:
decode(idx[0].tolist())

"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3"

### training

In [20]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

In [23]:
device = "cuda"
model = model.to(device)
# optimizer = optimizer.to(device)

In [29]:
batch_size = 32

for i in range(10000):
    xb, yb = get_batch("train")
    xb,yb = xb.to(device), yb.to(device)
    logits, loss= model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(loss.item())

2.729414939880371
2.608893394470215
2.5848662853240967
2.4988811016082764
2.4509129524230957
2.384185552597046
2.4576575756073
2.4058115482330322
2.39339280128479
2.5372300148010254


In [31]:
model.to("cpu")
idx = model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_token=100)
idx

tensor([[ 0, 16, 33, 15, 39, 42, 53, 53, 52, 48, 59,  1, 41, 47, 43,  1, 57, 58,
          1, 58,  1, 58, 56, 43, 43,  1, 21, 21, 27, 24, 17, 26, 30, 17, 32, 46,
         39, 52, 42,  1, 58,  6,  1, 39, 45, 46, 39, 58, 46,  6,  0, 25, 21, 10,
          1, 18, 53, 50,  5, 57, 58, 53, 56, 43,  1, 25, 21, 44,  1, 53, 57,  1,
         58, 46, 53,  1, 45, 43, 39, 58, 11,  1, 58, 47, 57, 46, 43, 43, 56,  1,
         50, 47, 42,  1, 39, 50,  5, 57, 58,  1, 40]])

In [32]:
decode(idx[0].tolist())

"\nDUCadoonju cie st t tree IIOLENREThand t, aghath,\nMI: Fol'store MIf os tho geat; tisheer lid al'st b"